# Fun stuff


In [2]:
import urllib2
import json
import unicodedata
import re

In [69]:
def fetch_characters():
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    title = "titles=List_of_Star_Wars_characters"
    content = "prop=revisions"
    rvprop ="rvprop=timestamp|content"
    dataformat = "format=json"
    rvdir = "rvdir=older" #sort revisions from newest to oldest
    end = "rvend=2000-01-03T00:00:00Z" #start of my time period
    start = "rvstart=2019-01-03T00:00:00Z" #end of my time period
    limit = "rvlimit=1" #consider only the first revision

    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % \
    (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    response = urllib2.urlopen(query)
    wikisource = response.read()
    wikijson = json.loads(wikisource)
    wikiid = wikijson["query"]["pages"].keys()[0]
    text = wikijson["query"]["pages"][wikiid]["revisions"][-1]["*"]
    # All characters on this page match the regex below
    regex = "\{\{visible anchor\|(.*?)\}\}"
    chars = re.findall(regex,text)
    characters = []
    for c in chars :
        for s in c.split("|") :
            st = ((s.replace("[", "")).replace("]","")).replace(" ","_")
            if (st.startswith("Admiral_")):
                st = st[8:]
            char = unicodedata.normalize('NFKD', st).encode('ascii','ignore')
            characters.append(char)   
    # Anakin skywalker is a special case
    characters.append("Anakin_Skywalker")
    return set(characters)


def fetch_wiki_article(title_):
    baseurl = "http://starwars.wikia.com/api.php?"
    action = "action=query"
    title = "titles=" + title_ + "&&redirects"
    content = "prop=revisions"
    rvprop ="rvprop=timestamp|content"
    dataformat = "format=json"
    rvdir = "rvdir=older" #sort revisions from newest to oldest
    end = "rvend=2000-01-03T00:00:00Z" #start of my time period
    start = "rvstart=2019-01-03T00:00:00Z" #end of my time period
    limit = "rvlimit=1" #consider only the first revision

    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % \
    (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    response = urllib2.urlopen(query)
    wikisource = response.read()
    wikijson = json.loads(wikisource)
    wikiid = wikijson["query"]["pages"].keys()[0]
    text = None
    # Below is equivalent to check if page exists
    if "revisions" in wikijson["query"]["pages"][wikiid].keys() :
        text = wikijson["query"]["pages"][wikiid]["revisions"][-1]["*"]
    
    return wikiid, text

In [70]:
charDict = {}
characters = fetch_characters()
wiki_ids = set()
for c in characters:
    wiki_id, article = fetch_wiki_article(c)
    if not article:
        continue
    if wiki_id not in wiki_ids:
        wiki_ids.add(wiki_id)
        charDict[c] = article
    else:
        print("DUPLICATE: %s" % c)
    

DUPLICATE: Grand_Moff_Tarkin
DUPLICATE: Grand_Moff_Wilhuff_Tarkin
DUPLICATE: L'ulo
DUPLICATE: Princess_Leia
DUPLICATE: Piett
DUPLICATE: General_Grievous
DUPLICATE: General_Hux
DUPLICATE: C1-10P
DUPLICATE: FN-2199
DUPLICATE: Grand_Admiral_Thrawn
DUPLICATE: Ben_Kenobi
DUPLICATE: R2-D2
DUPLICATE: Count_Dooku
DUPLICATE: Snoke
DUPLICATE: Gial_Ackbar
DUPLICATE: Doctor_Evazan
DUPLICATE: Jabba_the_Hutt
DUPLICATE: Jabba_Desilijic_Tiure
DUPLICATE: Kylo_Ren
DUPLICATE: Ozzel


In [72]:
CharDictKeysSorted = sorted (charDict.keys())
SortedCharacters = sorted (list (characters))
# Certain characters have a different name in their wookiepage, so we correct them manually
# Note to self: If they pop up on the wikipage, but is in the list of leftovers,
#    it is most likely because they are duplicates
leftovers = []
fixed_leftovers = []
for char in SortedCharacters:
    if char not in charDict:
        leftovers.append(char)
print("LIST OF CHARS NOT FOUND. HAS LENGTH: %s" % len(leftovers))
print(leftovers)





LIST OF CHARS NOT FOUND. HAS LENGTH: 61
['Aiolin_and_Morit_Astarte', 'Ben_Kenobi', 'Boss_Rugor_Nass', 'Breha_Antilles-Organa', 'C1-10P', 'Captain_Gregor', 'Captain_Quarsh_Panaka', 'Colonel_Kaplan', 'Commander_Cody_(CC-2224)', 'Commander_Cody_(Star_Wars)', 'Commander_Fil_(CC-3714)', 'Constable_Zuvio', 'Count_Dooku', 'Count_Dooku<br>{{small', 'Cpatain_Typho', 'Dinee_Ellberger', 'Doctor_Evazan', 'Dr._Cylo', 'FN-2199', 'Fifth_Brother_(Star_Wars)', 'Finn_(Star_Wars)', 'Garazeb_"Zeb"_Orrelios', 'General_Grievous', 'General_Hux', 'Gial_Ackbar', 'Grand_Admiral_Thrawn', 'Grand_Inquisitor_(Star_Wars)', 'Grand_Moff_Tarkin', 'Grand_Moff_Wilhuff_Tarkin', 'Has_Obbit', 'Inquisitors', 'Inspector_Thanoth', 'Jabba_Desilijic_Tiure', 'Jabba_the_Hutt', 'Kylo_Ren', "L'ulo", 'Lieutenant_Connix', 'Lieutenant_Kaydel_Ko_Connix', 'Lieutenant_Thire', 'Mister_Bones_(Star_Wars)', 'Niima_the_Hutt', 'Orrimaarko_("Prune_Face")', 'Orrimarko', 'Ozzel', 'Pagetti_Rook_("Weequay")', 'Petty_Officer_Thanisson', 'Piett', 'Pri